# Project Fatigue
- GitHub [link](https://github.com/romainmartinez/fatigue)

In [1]:
# Common imports
import scipy.io as sio
import numpy as np
import os

# Figures
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
sns.set_context("notebook", font_scale=1.1)
sns.set_style("ticks")
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42

# to make this notebook's output stable across runs
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

## 0. load data

In [2]:
DATA_PATH = os.path.join('.', 'data/')
DATA_FILE = 'DatabaseRPT.mat'
mat = sio.loadmat(os.path.join(DATA_PATH, DATA_FILE))['DataBaseRPT'][0, 0]

- label
    - `Y` (1, 162): 1 = prefatigue, 2 = fatigue

- used features
    - `AllX` (24, 162): 24 (6 DoF x 4 variables) x 162 (81 participants x 2 time points).
    - `CAssignAll` (1, 24): AllX column assignment

    - `Sex` (1, 162)
    - `Endurance` (1, 162)

- not used
    - `BestX` (7, 162): 7 (variables with SRM>0.8) x 162 (81 participants x 2 time points) matrix. Contains data only for the most responsive variables (SRM>0.8).
    - `CAssignBest` (1, 7): BestX column assignment.
    - `Age` (1, 162): too much NaN.
    - `Height` (1, 162): too much NaN.
    - `Weight` (1, 162) Too much NaN.
    - `SubjectKey` (1, 162): useless.
    - `SID` (1, 162): useless.

### create feature matrix

In [3]:
col_names = [i[0] for i in mat['CAssignAll'].flatten()]
# find variable with SRM > .8
srm_names = np.array([i[0] for i in mat['CAssignBest'].flatten()])
srm_idx = np.in1d(col_names, srm_names)

# add `AllX`
X = mat['AllX'].T

# add `Sex`
X = np.c_[X, mat['Sex'].T]
col_names.append('Sex')

# add `Endurance`
X = np.c_[X, mat['Endurance'].T]
col_names.append('Endurance')

col_names = np.array(col_names)

### create label vector

In [4]:
from sklearn.preprocessing import LabelBinarizer
y = mat['Y'].T
y = LabelBinarizer().fit_transform(y)

### create report

In [5]:
import os.path
import pandas as pd

# create dataframe
df = pd.DataFrame(X, columns=col_names)
df['fatigue (label)'] = y

# generate pandas report
REPORT_FILENAME = './pandas_report.html'
if not os.path.isfile(REPORT_FILENAME):
    import pandas_profiling
    report = pandas_profiling.ProfileReport(df)
    report.to_file('./pandas_report.html')

### split data & shuffle

In [6]:
# split data & shuffle
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

---

In [8]:
from tpot import TPOTClassifier

tpot = TPOTClassifier(generations=100, population_size=100, cv=5, scoring='roc_auc',
                      verbosity=2, n_jobs=-1, random_state=RANDOM_SEED)
tpot.fit(X_train, y_train.ravel())

/home/romain/.local/share/virtualenvs/fatigue-WgHAVgza/lib/python3.6/site-packages/deap/tools/_hypervolume/pyhv.py:33: ImportWarning: Falling back to the python version of hypervolume module. Expect this to be very slow.
  "module. Expect this to be very slow.", ImportWarning)
/home/romain/.local/share/virtualenvs/fatigue-WgHAVgza/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/home/romain/.local/share/virtualenvs/fatigue-WgHAVgza/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
Optimization Progress:   2%|▏         | 200/10100 [00:37<41:36,  3.97pipeline/s]

Generation 1 - Current best internal CV score: 0.9012820512820513


Optimization Progress:   3%|▎         | 300/10100 [01:10<59:21,  2.75pipeline/s]

Generation 2 - Current best internal CV score: 0.9012820512820513


Optimization Progress:   4%|▍         | 400/10100 [01:54<2:07:12,  1.27pipeline/s]

...

Generation 98 - Current best internal CV score: 0.9192307692307692


Optimization Progress:  99%|█████████▉| 10000/10100 [37:55<00:21,  4.59pipeline/s]

Generation 99 - Current best internal CV score: 0.9192307692307692


Generation 100 - Current best internal CV score: 0.9192307692307692

Best pipeline: GradientBoostingClassifier(input_matrix, learning_rate=0.001, max_depth=1, max_features=0.7500000000000001, min_samples_leaf=7, min_samples_split=16, n_estimators=100, subsample=0.3)


TPOTClassifier(config_dict={'sklearn.naive_bayes.GaussianNB': {}, 'sklearn.naive_bayes.BernoulliNB': {'alpha': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0], 'fit_prior': [True, False]}, 'sklearn.naive_bayes.MultinomialNB': {'alpha': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0], 'fit_prior': [True, False]}, 'sklearn.tree.DecisionT....3 , 0.35, 0.4 , 0.45, 0.5 , 0.55,
       0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95, 1.  ])}}}},
        crossover_rate=0.1, cv=5, disable_update_check=False,
        early_stop=None, generations=100, max_eval_time_mins=5,
        max_time_mins=None, memory=None, mutation_rate=0.9, n_jobs=4,
        offspring_size=100, periodic_checkpoint_folder=None,
        population_size=100, random_state=42, scoring=None, subsample=1.0,
        verbosity=2, warm_start=False)

In [9]:
print(tpot.score(X_test, y_test))

0.7867647058823529


In [10]:
tpot.export('tpot_pipeline.py')

True